# **Import Statements**

In [1]:
import sys
#root = "C:\\Users\\saman\\OneDrive\\Documents\\GitHub\\Eyelid-Modeling\\" #laptop path
root = "C:\\Users\\Samantha\\Documents\\GitHub\\Eyelid-Modeling\\" #desktop path
sys.path.append(root)

from utils.experiment_manager import Experiment
from models.EMG_RNN_CNN import EMG_RNN_CNN_Wrapper
from utils.data_management import Mat2TVT

import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle

%load_ext autoreload
%autoreload 2

torch.cuda.is_available()

True

# **Data Reorganization** <br>


In [2]:
eye_bool = np.asarray([True, True, False, True, True, False, False, False])
electrode_list = ['u1', 'u2', 'u3', 'u4', 't2']
marker_list = ['u1', 'u2', 'u3', 'u4', 'u5']
filepath = root + 'eyeliddata.mat'

train = 0.7
val = 0.2
test = 0.1

reorganizer = Mat2TVT(eye_bool, electrode_list, marker_list, filepath)
_, _, identifier = reorganizer.load_data()
X, y = reorganizer.DMVC_norm()
TVT_dict = reorganizer.TVT_split(train, val, test)

# **Run Experiments**

In [4]:
params = {
    't_win': np.arange(10,51,1, dtype = float),
    't_stride': np.arange(10, 51, 1, dtype = float),
    't_lookahead': np.arange(50, 101, 1, dtype = float),
    'win_mode': ['win_fft', 'win_rect'],
    'p_transform': np.linspace(0, 1, 100, dtype = float),
    'sigma': np.linspace(0, 5, 100, dtype = float),
    'p_mask': np.linspace(0, 0.5, 100, dtype = float),
    'batch_size': [4, 8, 16, 32, 64, 128],
    'learning_rate': np.linspace(1e-3, 1e-8, 500, dtype = float),
    'dropout': np.linspace(0, 1, 100, dtype = float),
    'RNN_hdim': [4, 8, 16, 32, 64, 128],
    'RNN_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'CNN_kernel': [3, 5]
}

experiment = Experiment()
model = EMG_RNN_CNN_Wrapper()
best_params = experiment(params, TVT_dict, model, n_trials = 250, epochs = 100, patience = 25, gcollect = False) 

print("Best Params:\n_________________________________________________________________________________________________________")
for key, value in best_params.items():
    print('    {}: {}'.format(key, value))

[I 2025-03-14 04:34:38,049] A new study created in memory with name: no-name-946390a3-c85f-4e8f-9bb3-f2a4f2424527


Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------Epoch 1
-------------------------------

Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Validation Error: 
 Max Marker Error: 3605.8%, Avg Marker Error: 172.5%, Avg loss: 1.331233 

Epoch 2
-------------------------------
Validation Error: 
 Max Marker Error: 6566.8%, Avg Marker Error: 183.6%, Avg loss: 1.621449 

Epoch 2
-------------------------------
Validation Error: 
 Max Marker Error: 3559.6%, Avg Marker Error: 171.3%, Avg loss: 1.322847 

Epoch 3
-------------------------------
Validation Error: 
 Max Marker Error: 4741.6%, Avg Marker Error: 135.8%, Avg loss: 1.274287 

Epoch 3
-------------------------------
Validation Error: 
 Max Marker Error: 3529.9%, Avg Marker Error: 170.0%, Avg loss: 1.314175 

Epoch 4
-------------------------------
Validation

[I 2025-03-14 04:40:45,243] Trial 4 finished with value: 123.96356201171875 and parameters: {'t_win': np.float64(24.0), 't_stride': np.float64(35.0), 't_lookahead': np.float64(67.0), 'win_mode': 'win_fft', 'p_transform': np.float64(0.30303030303030304), 'sigma': np.float64(4.545454545454545), 'p_mask': np.float64(0.4040404040404041), 'batch_size': 128, 'learning_rate': np.float64(0.00017636094188376746), 'dropout': np.float64(0.8787878787878789), 'RNN_hdim': 4, 'RNN_depth': 2, 'CNN_kernel': 3}. Best is trial 4 with value: 123.96356201171875.


Validation Error: 
 Max Marker Error: 1688.2%, Avg Marker Error: 124.0%, Avg loss: 1.080831 

Done!
Validation Error: 
 Max Marker Error: 1324.8%, Avg Marker Error: 95.3%, Avg loss: 1.025945 

Epoch 80
-------------------------------
Epoch 1
-------------------------------
Validation Error: 
 Max Marker Error: 1335.4%, Avg Marker Error: 95.3%, Avg loss: 1.024428 

Epoch 81
-------------------------------
Validation Error: 
 Max Marker Error: 1291.7%, Avg Marker Error: 95.8%, Avg loss: 1.050185 

Epoch 13
-------------------------------
Validation Error: 
 Max Marker Error: 1287.9%, Avg Marker Error: 95.3%, Avg loss: 1.025822 

Epoch 82
-------------------------------
Validation Error: 
 Max Marker Error: 1316.9%, Avg Marker Error: 95.3%, Avg loss: 1.025553 

Epoch 83
-------------------------------
Validation Error: 
 Max Marker Error: 700.8%, Avg Marker Error: 97.3%, Avg loss: 1.002370 

Epoch 23
-------------------------------
Validation Error: 
 Max Marker Error: 1328.0%, Avg Marker

[I 2025-03-14 04:42:24,937] Trial 0 finished with value: 95.23851013183594 and parameters: {'t_win': np.float64(21.0), 't_stride': np.float64(46.0), 't_lookahead': np.float64(97.0), 'win_mode': 'win_rect', 'p_transform': np.float64(0.15151515151515152), 'sigma': np.float64(1.3636363636363635), 'p_mask': np.float64(0.4696969696969697), 'batch_size': 64, 'learning_rate': np.float64(0.0008757527454909819), 'dropout': np.float64(0.3535353535353536), 'RNN_hdim': 16, 'RNN_depth': 10, 'CNN_kernel': 3}. Best is trial 0 with value: 95.23851013183594.


...

Validation Error: 
 Max Marker Error: 1422.4%, Avg Marker Error: 96.9%, Avg loss: 1.051017 

Validation Error: 
 Max Marker Error: 1500.6%, Avg Marker Error: 95.6%, Avg loss: 1.045259 

Epoch 27
-------------------------------
Validation Error: 
 Max Marker Error: 1653.7%, Avg Marker Error: 95.7%, Avg loss: 1.054121 

Epoch 23
-------------------------------
Validation Error: 
 Max Marker Error: 1559.2%, Avg Marker Error: 95.6%, Avg loss: 1.047130 

Epoch 28
-------------------------------


[I 2025-03-14 06:52:14,141] Trial 205 finished with value: 95.55126953125 and parameters: {'t_win': np.float64(44.0), 't_stride': np.float64(25.0), 't_lookahead': np.float64(50.0), 'win_mode': 'win_rect', 'p_transform': np.float64(0.6565656565656566), 'sigma': np.float64(1.0101010101010102), 'p_mask': np.float64(0.19191919191919193), 'batch_size': 8, 'learning_rate': np.float64(0.0004889830661322645), 'dropout': np.float64(0.8585858585858587), 'RNN_hdim': 32, 'RNN_depth': 7, 'CNN_kernel': 5}. Best is trial 84 with value: 92.28579711914062.


Validation Error: 
 Max Marker Error: 1525.9%, Avg Marker Error: 95.6%, Avg loss: 1.045483 

Stopped early after epoch: 27
Done!
Validation Error: 
 Max Marker Error: 1659.5%, Avg Marker Error: 95.7%, Avg loss: 1.054427 

Epoch 24
-------------------------------
Validation Error: 
 Max Marker Error: 1598.3%, Avg Marker Error: 95.7%, Avg loss: 1.054345 

Epoch 25
-------------------------------
Validation Error: 
 Max Marker Error: 1606.7%, Avg Marker Error: 95.7%, Avg loss: 1.053863 

Epoch 26
-------------------------------
Validation Error: 
 Max Marker Error: 1558.7%, Avg Marker Error: 95.7%, Avg loss: 1.053409 

Epoch 27
-------------------------------


[I 2025-03-14 06:55:05,130] Trial 87 finished with value: 95.6634521484375 and parameters: {'t_win': np.float64(46.0), 't_stride': np.float64(17.0), 't_lookahead': np.float64(50.0), 'win_mode': 'win_rect', 'p_transform': np.float64(0.8383838383838385), 'sigma': np.float64(0.5050505050505051), 'p_mask': np.float64(0.2777777777777778), 'batch_size': 4, 'learning_rate': np.float64(0.0005010069939879759), 'dropout': np.float64(0.6363636363636365), 'RNN_hdim': 16, 'RNN_depth': 9, 'CNN_kernel': 5}. Best is trial 84 with value: 92.28579711914062.


Validation Error: 
 Max Marker Error: 1547.4%, Avg Marker Error: 95.7%, Avg loss: 1.052976 

Stopped early after epoch: 26
Done!
Best Params:
_________________________________________________________________________________________________________
    t_win: 45.0
    t_stride: 44.0
    t_lookahead: 69.0
    win_mode: win_fft
    p_transform: 0.13131313131313133
    sigma: 4.242424242424242
    p_mask: 0.4191919191919192
    batch_size: 8
    learning_rate: 0.00042685943887775545
    dropout: 0.16161616161616163
    RNN_hdim: 64
    RNN_depth: 6
    CNN_kernel: 3


# **Training Model with Best Params**

In [ ]:
'''
Loading Stuff, to be used only if kernel crashes 
'''
model = EMG_RNN_CNN_Wrapper()
with open("best_params.pickle", "rb") as file:
    best_params = pickle.load(file)

metrics = experiment.run_experiment(best_params, TVT_dict, model, epochs = 1000, patience = 200)

'''
Saving Model, Training Metrics, and Best Params
'''
with open("training_metrics.pickle", "wb") as file:
    pickle.dump(metrics, file)
with open("best_params.pickle", "wb") as file:
    pickle.dump(best_params, file)

Epoch 1
-------------------------------
Validation Error: 
 Max Marker Error: 743.5%, Avg Marker Error: 98.4%, Avg loss: 1.052562 

Epoch 2
-------------------------------
Validation Error: 
 Max Marker Error: 798.3%, Avg Marker Error: 98.3%, Avg loss: 1.055276 

Epoch 3
-------------------------------
Validation Error: 
 Max Marker Error: 557.7%, Avg Marker Error: 98.4%, Avg loss: 1.048133 

Epoch 4
-------------------------------
Validation Error: 
 Max Marker Error: 712.3%, Avg Marker Error: 98.3%, Avg loss: 1.052027 

Epoch 5
-------------------------------
Validation Error: 
 Max Marker Error: 544.7%, Avg Marker Error: 98.4%, Avg loss: 1.047873 

Epoch 6
-------------------------------
Validation Error: 
 Max Marker Error: 543.3%, Avg Marker Error: 98.5%, Avg loss: 1.047963 

Epoch 7
-------------------------------
Validation Error: 
 Max Marker Error: 614.2%, Avg Marker Error: 98.5%, Avg loss: 1.049148 

Epoch 8
-------------------------------
Validation Error: 
 Max Marker Error

: 

# **Plotting Loss Metrics**

In [ ]:
plt.plot(metrics['Training Loss'], label = 'Training Loss', marker = '.')
plt.plot(metrics['Validation Loss'], label = 'Validation Loss', marker = '.')
plt.yscale('log')
plt.title('Loss vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(metrics['Training Avg Marker Error'], label = 'Training Error', marker = '.')
plt.plot(metrics['Validation Avg Marker Error'], label = 'Validation Error', marker = '.')
plt.yscale('log')
plt.title('Average Marker Error vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Average Marker Error')
plt.legend()
plt.show()

plt.plot(metrics['Training Max Marker Error'], label = 'Training Error', marker = '.')
plt.plot(metrics['Validation Max Marker Error'], label = 'Validation Error', marker = '.')
plt.yscale('log')
plt.title('Max Marker Error vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Max Marker Error')
plt.legend()
plt.show()